In [23]:
import pandas as pd
import json
import datetime
import glob
import numpy as np

In [24]:
Store_Df=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2018-07-28.csv",dtype=str)
Store_Df['week_end_date']=Store_Df['week_end_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
Store_Df=Store_Df[(Store_Df['week_end_date']>=datetime.date(2018,2,1))&(Store_Df['week_end_date']<=datetime.date(2018,5,6))]
Store_Df['sales']=Store_Df['sales'].astype(float)
Store_Df=Store_Df[Store_Df['sales']>0]

store_list_Q1=Store_Df['location_id'].unique().tolist()

In [25]:
store_list_1=pd.read_table("/home/jian/BiglotsCode/OtherInput/MediaStormStoreList_Nov15.txt",dtype=str,sep="|")
store_list_2=pd.read_table("/home/jian/BiglotsCode/OtherInput/MediaStormStores_20180703.txt",dtype=str,sep="|")
store_list_1['store_zip']=store_list_1['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
store_list_2['store_zip']=store_list_2['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))

store_list_1=store_list_1[['location_id','store_zip','longitude_meas','latitude_meas']]
store_list_2=store_list_2[['location_id','store_zip','longitude_meas','latitude_meas']]
store_list_1=store_list_1[~store_list_1['location_id'].isin(store_list_2['location_id'])]
store_list=store_list_2.append(store_list_1)
store_list=store_list[store_list['location_id']!="6990"]
store_list=store_list[store_list['location_id']!="145"]


store_list['latitude_meas']=store_list['latitude_meas'].astype(float)
store_list['longitude_meas']=store_list['longitude_meas'].astype(float)

In [26]:
store_list=store_list[store_list['location_id'].isin(store_list_Q1)]
inclued_zips=store_list['store_zip'].unique().tolist()

In [27]:
weather_json_files=glob.glob("/home/jian/Projects/Big_Lots/Weather/Json_data/daily/"+"*.json")
weather_date=[datetime.datetime.strptime(x.split("/")[len(x.split("/"))-1][0:10],"%Y-%m-%d").date() for x in weather_json_files]
df_weather_files=pd.DataFrame({"path":weather_json_files,"date":weather_date},index=range(len(weather_json_files)))
df_weather_files=df_weather_files[(df_weather_files['date']>=datetime.date(2018,2,1)) & (df_weather_files['date']<datetime.date(2018,3,1))]


In [28]:
len(df_weather_files['date'].unique())

27

In [29]:
group_weight=pd.read_excel("/home/jian/Projects/Big_Lots/Weather/Q1_Weather_Counts/Q1_inclusion_store_all_weather_type_ranked.xlsx",sheetname="all_weather_group_list")
group_weight['Severity']=group_weight['Severity'].astype(int)
group_weight=group_weight[['all_type_group','Severity']]
group_weight_dict=group_weight[['all_type_group', 'Severity']].set_index('all_type_group').T.to_dict()

In [34]:
output_1_month=pd.DataFrame()
for path in df_weather_files['path'].tolist():
    response=json.load(open(path,"r"))
    DATE=df_weather_files[df_weather_files['path']==path]['date'].unique()[0]
    output_1_Day=pd.DataFrame()
    for zip_cd in inclued_zips:
        if zip_cd in list(response.keys()):
            weather_list=response[zip_cd]['weather']
            dominated_weather_severity=0
            dominated_weather=np.nan
            for j in range(len(weather_list)):
                weather_j=weather_list[j]['main']
                weather_j_severity=group_weight_dict[weather_j]['Severity']
                if weather_j_severity>dominated_weather_severity:
                    dominated_weather=weather_j
                    dominated_weather_severity=weather_j_severity
            df_app=pd.DataFrame({"Date":datetime.datetime.fromtimestamp(response[list(response.keys())[0]]['dt']).date(),"zip_cd":zip_cd,"Weather":dominated_weather,'Severity':dominated_weather_severity},index=[0])
            output_1_Day=output_1_Day.append(df_app)
            output_1_Day['file_date']=DATE
    output_1_month=output_1_month.append(output_1_Day)

In [37]:
zip_DMA=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1)
zip_DMA=zip_DMA.iloc[:,[0,2]]
zip_DMA.columns=['zip_cd','DMA']
zip_DMA=zip_DMA.drop_duplicates()

In [40]:
Feb_Severity_DMA_Duplicated=pd.merge(output_1_month,zip_DMA,on="zip_cd",how="left")
Feb_Severity_DMA_Duplicated=Feb_Severity_DMA_Duplicated.groupby(['DMA'])['Severity'].mean().to_frame().reset_index().sort_values(['Severity'],ascending=False)


In [47]:
zip_count_DMA=pd.merge(output_1_month,zip_DMA,on="zip_cd",how="left")
zip_count_DMA=zip_count_DMA[zip_count_DMA['file_date']==zip_count_DMA['file_date'].max()]
zip_count_DMA=zip_count_DMA.groupby(['DMA'])['zip_cd'].count().to_frame().reset_index().rename(columns={"zip_cd":"zip_count"})
output=pd.merge(Feb_Severity_DMA_Duplicated,zip_count_DMA,on="DMA",how="left")

In [49]:
output.to_csv("/home/jian/Projects/Big_Lots/Weather/Internal_Search_Phrase/Feb_27_Days_Severity_by_DMA_JL_20180806.csv",index=False)